## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 데이터 전처리 마무리 하기


In [2]:
# 필요라이브러리 사용 등록
import pandas as pd
import folium
import json

In [76]:
# 서울시의 스타벅스 매장 정보 로드하기
dfStarbukcsSeoul = pd.read_excel('./data/starbukcs_seoul_list.xlsx')


In [6]:
# 주소를 split으로 자른 다음에 배열의 1번 째 인덱스가 구이다.
# 기존 DataFrame에 구 추가하기
guName = []
for number in range(0,len(dfStarbukcsSeoul['주소'])):
    gu = dfStarbukcsSeoul['주소'][number].split(' ')[1]
    guName.append(gu)
df_guName = pd.DataFrame(guName)
dfStarbukcsSeoul['구'] = df_guName

In [7]:
dfStarbukcsSeoul.tail()

,매장명,위도,경도,매장 타입,주소,전화번호,구
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232,중랑구
609,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,중랑구
611,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,중랑구
612,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35 1층,1522-3232,중랑구


In [21]:
dfStarbukcsSeoul.to_excel('./data/최종_서울_스타벅스매장리스트.xlsx',index=False)

In [8]:
# 스타벅스 분석 데이터 만들기
dfSeoulGuList = pd.read_excel('./data/수정_서울_시군구_위경도.xlsx')

In [23]:
dfSeoulGuList.tail()

,docity,do,city,longitude,latitude
20,서울용산구,서울,용산구,126.967522,37.536094
21,서울은평구,서울,은평구,126.931242,37.599969
22,서울종로구,서울,종로구,126.981642,37.570378
23,서울중구,서울,중구,126.999642,37.561003
24,서울중랑구,서울,중랑구,127.094778,37.603806


In [12]:
dfSeoulGuLast = dfSeoulGuList[['do','city','latitude','longitude']]

In [13]:
# 각 칼럼의 영어이름은 한글 이름으로 변경
dfSeoulGuLast.columns = ['시','구','위도','경도']

In [14]:
dfSeoulGuLast.tail()

,시,구,위도,경도
20,서울,용산구,37.536094,126.967522
21,서울,은평구,37.599969,126.931242
22,서울,종로구,37.570378,126.981642
23,서울,중구,37.561003,126.999642
24,서울,중랑구,37.603806,127.094778


In [34]:
dfSeoulGuLast.to_excel('./data/최종_서울_구리스트.xlsx',index=False)

In [10]:
# 서울시 스타벅스 매장리스트에서 구별 데이터 합산(pivot_table)
seoulStarbukcsGuCount = dfStarbukcsSeoul.pivot_table(values='매장명',index='구',aggfunc='count').rename(columns={'매장명':'매장수'})

In [11]:
seoulStarbukcsGuCount.tail()

,매장수
구,
용산구,24
은평구,14
종로구,40
중구,54
중랑구,8


In [16]:
# 서울구 리스트에 스타벅스구별 매장수를 병합 last를 앞이 붙이면 진짜 최종 데이터!!!!
lastSbSeoulCount = pd.merge(left=dfSeoulGuLast,right=seoulStarbukcsGuCount,how='left',on='구')

In [20]:
#서울시 구별 인구통계 로드하기
dfSeoulGuPop = pd.read_excel('./data/수정_서울시_구별_인구데이터.xlsx',usecols=['구별','소계'])
dfSeoulGuPop.tail()

,구별,소계
21,관악구,497883
22,서초구,412078
23,강남구,550282
24,송파구,660025
25,강동구,463318


In [ ]:
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount,right=dfSeoulGuPop,how='left',left_on='구',right_on='구별').drop(['구별_x','소계_x','구별_y','소계_y'],axis=1)

In [43]:
lastSbSeoulCount.columns = ['시','구','위도','경도','스차벅스매장수','구별','주민등록인구']
lastSbSeoulCount.drop('구별',axis=1,inplace=True)

In [45]:
lastSbSeoulCount.tail()

,시,구,위도,경도,스차벅스매장수,주민등록인구
20,서울,용산구,37.536094,126.967522,24,227106
21,서울,은평구,37.599969,126.931242,14,470869
22,서울,종로구,37.570378,126.981642,40,150453
23,서울,중구,37.561003,126.999642,54,131793
24,서울,중랑구,37.603806,127.094778,8,387470


In [46]:
dfSeoulGuBiz = pd.read_excel('./data/수정_서울_구별사업체.xlsx',usecols=['구','소계','종사자수 소계'])
dfSeoulGuBiz.tail()

,구,소계,종사자수 소계
21,관악구,37915,128417
22,서초구,71958,487976
23,강남구,110007,801419
24,송파구,74531,400781
25,강동구,41660,157465


In [48]:
## 사업체 정보에서 컬럼이름부터 변경
dfSeoulGuBiz.columns = ['구','사업체수','종사자수']
dfSeoulGuBiz.tail()

,구,사업체수,종사자수
21,관악구,37915,128417
22,서초구,71958,487976
23,강남구,110007,801419
24,송파구,74531,400781
25,강동구,41660,157465


In [50]:
# 사업체 정보를 스타벅스 최종 데이터에 병합
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount,right=dfSeoulGuBiz,how='left',on='구')

In [52]:
lastSbSeoulCount.to_excel('./data/최종_서울_스타벅스매장_데이터.xlsx',index=False)

##### 스타벅스 분석 시각화

In [6]:
dfStarbucksSeoul = pd.read_excel('./data/최종_서울_스타벅스매장리스트.xlsx')
dfStarbucksSeoul.tail()

,매장명,위도,경도,매장 타입,주소,전화번호,구
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232,중랑구
609,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,중랑구
611,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,중랑구
612,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35 1층,1522-3232,중랑구


In [11]:
dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')
dfStarbucksSeoulCount.tail()

,시,구,위도,경도,스타벅스매장수,주민등록인구,사업체수,종사자수
20,서울,용산구,37.536094,126.967522,24,227106,29680,152605
21,서울,은평구,37.599969,126.931242,14,470869,36509,109031
22,서울,종로구,37.570378,126.981642,40,150453,48361,275063
23,서울,중구,37.561003,126.999642,54,131793,70308,386564
24,서울,중랑구,37.603806,127.094778,8,387470,39310,113144


In [23]:
#서울시 지도 생성
seoulLoc = (37.573050,126.979169)
seoulMap = folium.Map(location=(37.573050,126.979169),zoom_start=11)

# 각 매장별 위치를 지도에 마커로 표시
for idx in range(0,len(dfStarbucksSeoul)):
    lat = dfStarbucksSeoul.loc[idx,'위도']
    lng = dfStarbucksSeoul.loc[idx,'경도']
    type = dfStarbucksSeoul.loc[idx,'매장 타입']
    if type == 'general':
        folium.CircleMarker(location=(lat,lng),popup=f'{dfStarbucksSeoul.loc[idx,'매장명']}점',fill=True,fill_color='green',
                        fill_opacity=2,color='yellow',radius=4,weight=1).add_to(seoulMap)
    elif type == 'reserve':
        folium.CircleMarker(location=(lat,lng),popup=f'{dfStarbucksSeoul.loc[idx,'매장명']}점',fill=True,fill_color='black',
                        fill_opacity=2,color='yellow',radius=4,weight=1).add_to(seoulMap)
    elif type == 'generalDT':
        folium.CircleMarker(location=(lat,lng),popup=f'{dfStarbucksSeoul.loc[idx,'매장명']}점',fill=True,fill_color='red',
                        fill_opacity=2,color='yellow',radius=4,weight=1).add_to(seoulMap)
    elif type == 'generalWT':
        folium.CircleMarker(location=(lat,lng),popup=f'{dfStarbucksSeoul.loc[idx,'매장명']}점',fill=True,fill_color='violet',
                        fill_opacity=2,color='yellow',radius=4,weight=1).add_to(seoulMap)

seoulMap

In [24]:
#매장 타입 개수
dfStarbucksSeoul['매장 타입'].value_counts()

매장 타입
general      552
reserve       36
generalDT     23
generalWT      2
Name: count, dtype: int64

In [32]:
seoulLoc = (37.573050,126.979169)
sbMap = folium.Map(location=(37.573050,126.979169),zoom_start=11)

# 각 매장별 위치를 지도에 마커로 표시
for idx in range(0,len(dfStarbucksSeoul)):
    lat = dfStarbucksSeoul.loc[idx,'위도']
    lng = dfStarbucksSeoul.loc[idx,'경도']
    store_type = dfStarbucksSeoul.loc[idx,'매장 타입']

    # 매장 타입별 색상
    fillColor = ''
    if store_type == 'general':
        fillColor = 'darkgreen'
        size = 3
    elif store_type == 'reserve':
        fillColor = 'black'
        size = 5
    elif store_type == 'generalDT':
        fillColor = 'orange'
        size = 3
    elif store_type == 'generalWT':
        fillColor = 'purple'
        size = 4
    folium.CircleMarker(location=(lat,lng),popup=f'{dfStarbucksSeoul.loc[idx,'매장명']}점',fill=True,fill_color=fillColor,
                fill_opacity=2,color='yellow',radius=size,weight=1).add_to(sbMap)
    
sbMap

In [40]:
# 서울시 구별 스타벅스 매장 수 시각화

dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')
dfStarbucksSeoulCount.tail()

,시,구,위도,경도,스타벅스매장수,주민등록인구,사업체수,종사자수
20,서울,용산구,37.536094,126.967522,24,227106,29680,152605
21,서울,은평구,37.599969,126.931242,14,470869,36509,109031
22,서울,종로구,37.570378,126.981642,40,150453,48361,275063
23,서울,중구,37.561003,126.999642,54,131793,70308,386564
24,서울,중랑구,37.603806,127.094778,8,387470,39310,113144


In [5]:
path = './data/seoul_municipalities_geo_simple.json'

In [6]:
# 서울시 행정구역(구별) 경계지도 그리기
seoulGeoSimple = json.load(open(path,encoding='utf-8'))

In [ ]:
seoulGeoSimple

In [7]:
seoulGeoSimple['features'][0]['properties']

{'code': '11250',
 'name': '강동구',
 'name_eng': 'Gangdong-gu',
 'base_year': '2013'}

In [9]:
def style_function(feature):
    return {
        'opacity' : 0.9,
        'weight' : 2,
        'color' : 'white',
        'fillOpacity' : 0,
        'dashArray' : '5 , 3',

    }

In [ ]:
## 지도 생성
seoulLoc = (37.573050,126.979169)
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11,tiles='CartoDb dark_matter')
folium.GeoJson(seoulGeoSimple,style_function).add_to(sbCountMap)


In [41]:
## 스타벅스 평균 매장 수 계산 = 24.52 < 구별로 평균 25개의 매장이 존재한다.
sbMean = dfStarbucksSeoulCount['스타벅스매장수'].mean()

In [49]:
for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']


    if count > sbMean:
        fillColor = 'red' #평균보다 크면 빨간색
    else: fillColor = 'blue'

    folium.CircleMarker(location=(lat,lng),color='#FFFF00',fill_color=fillColor,radius=count/2,
                        fill_opacity=0.7,weight=1.5,popup=guName + str(count)).add_to(sbCountMap)
    

In [15]:
from folium.features import DivIcon # hml로 텍스트 표시할 아이콘 모듈 추가

In [87]:
## 지도 생성 구별 스타벅스 매장수 시각화
seoulLoc = (37.573050,126.979169)
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11)
folium.GeoJson(seoulGeoSimple,style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple,data=dfStarbucksSeoulCount,columns=['구','스타벅스매장수'],
                  legend_name='구별 스타벅스 매장 수',
                  fill_color='YlGn',fill_opacity=0.7,line_opacity=0.5,key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']
    # 마커와 DivIcon으로 지도에 텍스트 표시
    folium.Marker(location=(lat,lng),icon=DivIcon(
        icon_size = (150,60),
        icon_anchor = (75,25),
        html = f'''
        <div style="font-size: 9pt; font-weight: bold; color:black;text-align: center;">
        {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)

sbCountMap


In [ ]:
for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']


    if count > sbMean:
        fillColor = 'red' #평균보다 크면 빨간색
    else: fillColor = 'blue'

    folium.CircleMarker(location=(lat,lng),color='#FFFF00',fill_color=fillColor,radius=count/2,
                        fill_opacity=0.7,weight=1.5,popup=guName + str(count)).add_to(sbCountMap)
    
sbCountMap

In [18]:
## 서울시 구별 인구수 지도 시각화
seoulLoc = (37.573050,126.979169)
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11)
folium.GeoJson(seoulGeoSimple,style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple,data=dfStarbucksSeoulCount,columns=['구','주민등록인구'],
                  legend_name='구별 인구분포',
                  fill_color='YlOrRd',fill_opacity=0.7,line_opacity=0.5,key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']
    pop = dfStarbucksSeoulCount.loc[idx,'주민등록인구']
    # 마커와 DivIcon으로 지도에 텍스트 표시
    folium.Marker(location=(lat,lng),icon=DivIcon(
        icon_size = (150,60),
        icon_anchor = (75,25),
        html = f'''
        <div style="font-size: 9pt; font-weight: bold; color:black;text-align: center;">
        {guName} / {pop}
        </div>
        '''
    )).add_to(sbCountMap)

sbCountMap

In [22]:
## 서울시 구별 사업체 지도 시각화
seoulLoc = (37.573050,126.979169)
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11)
folium.GeoJson(seoulGeoSimple,style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple,data=dfStarbucksSeoulCount,columns=['구','사업체수'],
                  legend_name='구별 사업체분포',
                  fill_color='PuBu',fill_opacity=0.7,line_opacity=0.5,key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'사업체수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']
    pop = dfStarbucksSeoulCount.loc[idx,'주민등록인구']
    # 마커와 DivIcon으로 지도에 텍스트 표시
    folium.Marker(location=(lat,lng),icon=DivIcon(
        icon_size = (150,60),
        icon_anchor = (75,25),
        html = f'''
        <div style="font-size: 9pt; font-weight: bold; color:black;text-align: center;">
        {guName} / {pop}
        </div>
        '''
    )).add_to(sbCountMap)

sbCountMap